In [97]:
import pandas as pd
from compas import Compas
import torch
from Models import MLP

In [41]:
d = Compas()

In [83]:
cfs_df = pd.read_csv('human_feedback.csv')
instances_df = pd.read_csv('instances_with_id.csv', header=None)
cfs_np = cfs_df.to_numpy()
instances_np = instances_df.to_numpy()
cfs_torch = d.df_to_torch(cfs_df.iloc[:, 2:-2])
instances_torch = instances_df.iloc[:, 1:-1].copy()
instances_torch.columns = d.column_names
instances_torch = d.df_to_torch(instances_torch)

In [91]:
n_pairs = 1305
n_columns = d.data_torch_train.shape[1]

pairs = torch.zeros(n_pairs, n_columns)
targets = torch.zeros(n_pairs)

counter = 0
nr = 0

for i in range(len(instances_np)-1):
    i_id = instances_np[i][0]
    i_torch = instances_torch[i]
    for cf in range(5):
        
        if cfs_np[counter][0] != i_id:
            print('Something goes wrong')
        
        if cfs_np[counter][-1] != 'None':
            
            cf_torch = cfs_torch[counter]
            difference = cf_torch - i_torch
            
            pairs[nr] = difference
            
            if cfs_np[counter][-1] == 'True':
                targets[nr] = 0
            else:
                targets[nr] = 1
                
            nr += 1
                
    
            
        counter +=1
            

## 0 means fair and 1 means unfair

In [95]:
sum(targets)/len(targets)

tensor(0.3655)

In [96]:
split = round(0.75 * n_pairs)
x_train = pairs[:split].detach()
x_test = pairs[split:].detach()
y_train = targets[:split].detach()
y_test = targets[split:].detach()

In [98]:
input_dim = len(x_train[0])
hidden_dim = 100

mlp_model = MLP(input_dim, hidden_dim)
criterion = torch.nn.BCELoss() # BCE = binary cross entropy - our targets are binary 
optimizer = torch.optim.Adam(mlp_model.parameters(), lr = 0.001)

### EVAL ###
mlp_model.eval() # here sets the PyTorch module to evaluation mode. 
y_train_hat = mlp_model(x_train)
before_train = criterion(y_train_hat.squeeze(), y_train)
print('Test loss before training' , before_train.item())

### TRAIN ###
mlp_model.train() # here sets the PyTorch module to train mode. 
tot_epoch = 401
for epoch in range(tot_epoch):
    optimizer.zero_grad()
    # Forward pass
    y_train_hat = mlp_model(x_train)
    # Compute Loss
    loss = criterion(y_train_hat.squeeze(), y_train)
    
    if epoch%100==0:
        y_test_hat = mlp_model(x_test)
        print('Epoch: {} -- train loss: {} -- accuracy (test set): {}'.format(epoch, round(loss.item(), 3), mlp_model.accuracy(y_test_hat, y_test)))
        y_test_hat = mlp_model(x_test)
        
    # Backward pass
    loss.backward()
    optimizer.step()

### EVAL ###
mlp_model.eval()
y_test_hat = mlp_model(x_test)
after_train = criterion(y_test_hat.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())

print('The accuracy scrore on the test set:', mlp_model.accuracy(y_test_hat, y_test))

Test loss before training 0.688576877117157
Epoch: 0 -- train loss: 0.689 -- accuracy (test set): 0.58
Epoch: 100 -- train loss: 0.589 -- accuracy (test set): 0.678
Epoch: 200 -- train loss: 0.541 -- accuracy (test set): 0.69
Epoch: 300 -- train loss: 0.516 -- accuracy (test set): 0.687
Epoch: 400 -- train loss: 0.494 -- accuracy (test set): 0.681
Test loss after Training 0.520545482635498
The accuracy scrore on the test set: 0.681


In [99]:
torch.save(mlp_model, "f_fair_compas_human.pt")